Combining All History JSON Files

In [2]:
import pandas as pd
import time
#combining total history
history0 = pd.read_json('files/syd0.json')
history1 = pd.read_json('files/syd1.json')
history2 = pd.read_json('files/syd2.json')
history3 = pd.read_json('files/syd3.json')
history4 = pd.read_json('files/syd4.json')
history5 = pd.read_json('files/syd5.json')
history6 = pd.read_json('files/syd6.json')
history7 = pd.read_json('files/syd7.json')
history8 = pd.read_json('files/syd8.json')
history9 = pd.read_json('files/syd9.json')
history10 = pd.read_json('files/syd10.json')
history11 = pd.read_json('files/syd11.json')
history12 = pd.read_json('files/syd12.json')
history13 = pd.read_json('files/syd13.json')
history14 = pd.read_json('files/syd14.json')
history15 = pd.read_json('files/syd15.json')
history16 = pd.read_json('files/syd16.json')
history17 = pd.read_json('files/syd17.json')
history18 = pd.read_json('files/syd18.json')
history19 = pd.read_json('files/syd19.json')
complete = pd.concat([history0,history1,history2,history3,history4,history5,history6,history7,history8,history9,history10,history11,history12,history13,history14,history15,history16,history17,history18,history19])
print(complete.columns)







Index(['ts', 'platform', 'ms_played', 'conn_country', 'ip_addr',
       'master_metadata_track_name', 'master_metadata_album_artist_name',
       'master_metadata_album_album_name', 'spotify_track_uri', 'episode_name',
       'episode_show_name', 'spotify_episode_uri', 'audiobook_title',
       'audiobook_uri', 'audiobook_chapter_uri', 'audiobook_chapter_title',
       'reason_start', 'reason_end', 'shuffle', 'skipped', 'offline',
       'offline_timestamp', 'incognito_mode'],
      dtype='object')


Cleaning and Reformatting/naming Columns

In [3]:
#Drop unecessary columns and rename original
complete = complete.drop(columns=['platform', 'ip_addr', 'audiobook_title', 'audiobook_uri', 'audiobook_chapter_uri', 'audiobook_chapter_title', 'offline', 'offline_timestamp'])
complete = complete.rename(columns={'ts':'exact_time', 'ms_played':'milliseconds', 'conn_country':'country','master_metadata_track_name':'track_name',	'master_metadata_album_artist_name':'artist_name','master_metadata_album_album_name':'album_name', 'spotify_track_uri':'track_uri', 'episode_name':'episode_name','episode_show_name':'show_name', 'spotify_episode_uri':'episode_uri','reason_start':'start_reason','reason_end':'end_reason','incognito_mode':'incognito'})

#convert ms to minutes
complete['minutes'] = complete['milliseconds']/60000
#extract specific times
complete['exact_time'] = pd.to_datetime(complete['exact_time'])
complete['hour'] = complete['exact_time'].dt.hour
complete['day'] = complete['exact_time'].dt.date
complete['month'] = complete['exact_time'].dt.month
complete['weekday'] = complete['exact_time'].dt.weekday
complete['year'] = complete['exact_time'].dt.year

complete.head() # complete['year'] = complete['year'].dt.year


,exact_time,milliseconds,country,track_name,artist_name,album_name,track_uri,episode_name,show_name,episode_uri,...,end_reason,shuffle,skipped,incognito,minutes,hour,day,month,weekday,year
0,2020-04-06 20:27:23+00:00,21873,US,ball w/o you,21 Savage,i am > i was,spotify:track:50a8bKqlwDEqeiEknrzkTO,None,None,None,...,endplay,False,False,False,0.364550,20,2020-04-06,4,0,2020
1,2020-04-06 20:29:19+00:00,1830,US,Far From You,Rich The Kid,BOSS MAN,spotify:track:4VV8svAvJBYCPRTRwl01Z5,None,None,None,...,endplay,False,False,False,0.030500,20,2020-04-06,4,0,2020
2,2020-04-06 20:29:23+00:00,3950,US,Stuck Together (feat. Lil Baby),Rich The Kid,BOSS MAN,spotify:track:5cHPAqi6V5JPMy0vqS4lK6,None,None,None,...,endplay,False,False,False,0.065833,20,2020-04-06,4,0,2020
3,2020-04-06 20:29:29+00:00,5700,US,Sick (feat. DaBaby),Rich The Kid,BOSS MAN,spotify:track:0BqyyCV8M6roYLRmNtiasC,None,None,None,...,endplay,False,False,False,0.095000,20,2020-04-06,4,0,2020
4,2020-04-06 20:29:37+00:00,7420,US,Not Sorry (feat. Nicki Minaj),Rich The Kid,BOSS MAN,spotify:track:69pWehVZg0u3o17imQuEVj,None,None,None,...,endplay,False,False,False,0.123667,20,2020-04-06,4,0,2020


Split Tracks and Episodes

In [4]:
#Split up dataframe into tracks and episodes/podcasts

episodes = complete[complete['episode_name'].notnull()] #save rows where there's an episodeName
tracks = complete[complete['track_name'].notnull()] #save rows where there's a trackName
print(len(episodes))
print(len(tracks))
print(len(complete))
#tracks = complete[~complete['trackName'].notnull()]



616
304945
305562


Store all rows with unique artists

In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
from dotenv import load_dotenv
import os
client_id = os.getenv('client_id')
client_secret = os.getenv('client_secret')
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))
#only keep unique artists
uniqueArtists = tracks.drop_duplicates(subset=['artist_name'])

print(len(uniqueArtists))
print(os.getenv('client_id'))



5254
46355142896741e6aeb23664375a1a2c


Connect to PostgreSQL DB

In [ ]:

# from sqlalchemy import create_engine
# from sqlalchemy import inspect
# import psycopg2

#  #connection to postgres db
# conn_string = 'postgresql://postgres:' + os.getenv('postgres_pass')  + '@localhost:' + os.getenv('postgres_port') + '/Spotify'
# db = create_engine(conn_string)
# #validate if connection works
# inspector = inspect(db)
# print(inspector.get_table_names(schema='public'))
# conn = db.connect()






Populate table in postgres with track history

In [ ]:

# #Transfer tracks rows to sql table trackhistory
# tracks.to_sql('trackhistory', con=conn, if_exists='replace', index=False)
# #Validate rows
# print(pd.read_sql("SELECT COUNT(*) FROM trackhistory;", con=engine))




In [23]:
# import time
import numpy as np
artistGenre = {}

# #Extract all unique artist IDs and save in list
# artist_ids = []
# for t in uniqueArtists['track_uri']:
#     try:
#         trackInfo = sp.track(t)
#         artist = trackInfo['artists'][0]
#         if artist['id'] not in artist_ids:
#             artist_ids.append(artist['id'])
#     except Exception as e:
#         print(e)

# #Fetch artists in batches of 50
# batch_size = 50
# for i in range(0, len(artist_ids), batch_size):
#     batch = artist_ids[i:i + batch_size]
#     try:
#         # Batch request
#         artists_data = sp.artists(batch)['artists']
#         for a in artists_data:
#             name = a['name']
#             genres = a.get('genres', [])
#             artistGenre[name] = genres

#         #Small buffer to stay under rate limit
#         time.sleep(0.2)
#     except Exception as e:
#         print(f"Error fetching batch starting at index {i}: {e}")
#         time.sleep(5)
#         continue

uniqueArtistsSplit = np.array_split(uniqueArtists, 6)
print(len(uniqueArtistsSplit))







6


c:\Users\sydne\anaconda3\envs\spotify-analysis\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:


# trackInfo = sp.track("spotify:track:50a8bKqlwDEqeiEknrzkTO") #Get spotify trackinfo
# artist = trackInfo['artists'][0] 
# artistID = artist['id'] #Get artist ID from artist objet
# genres =  sp.artist(artistID)['genres'] #Get genre(s) of artists
# print(genres)
#Dict to map artist with genres

for t in uniqueArtistsSplit[2]['track_uri']:
    try: 
        trackInfo = sp.track(t) #Get spotify trackinfo
        artist = trackInfo['artists'][0] 
        artistID = artist['id'] #Get artist ID from artist objet
        genres =  sp.artist(artistID)['genres'] #Get genre(s) of artists
        if artist['name'] not in artistGenre:
            artistGenre[artist['name']] = genres
        time.sleep(0.1)
    except Exception as e: #If artist is invalid print error  ``
        print(e)
        continue


print(len(artistGenre))


# #print(top)
# # print(client_id)
# # print(client_secret)

2


In [ ]:
print(artistGenre)
theDict = pd.DataFrame(artistGenre.items(), columns=["artist", "genres"])
theDict = theDict.explode("genres")
theDict['genres'] = theDict['genres'].fillna("No Genre")
output = "artistGenre.csv"
theDict.to_csv(output, mode ='a', header=False, index = False, encoding = "utf-8")





{'$NOT': [], 'Josh Turner': ['country', 'christian country'], 'Luke Combs': ['country'], 'Drew Holcomb & The Neighbors': ['christian folk'], 'Dylan Scott': ['country'], 'Elton John': [], 'Jordan Smith': ['christmas'], 'Chicago': ['yacht rock', 'soft rock'], 'Caroline Glaser': ['folk pop'], 'Cassadee Pope': [], 'Chris Jamison': [], 'Deanna Johnson': [], 'Big Boss Vette': [], 'Benny Benassi': ['electro house'], 'JP Saxe': [], 'Ashe': [], 'Jake Miller': [], 'The Cinematic Orchestra': ['nu jazz', 'downtempo'], 'Kina': [], 'TELYKAST': [], 'Ir Sais': ['kizomba'], 'Dennis Kruissen': [], 'Jordin Sparks': [], 'Princess Nokia': [], 'Mighty Bay': [], 'King Critical': [], 'John Newman': [], 'potsu': ['lo-fi'], 'Maxine Nightingale': [], 'The Lonely Island': ['comedy'], 'Bobby Darin': ['big band', 'adult standards', 'swing music', 'doo-wop'], 'The Beatles': ['classic rock', 'psychedelic rock'], 'Tears For Fears': ['new wave', 'synthpop'], 'Backstreet Boys': [], 'Oasis': ['britpop', 'madchester'], '*